In [1]:
import numpy as np
from numpy.linalg import lstsq

In [2]:
a = np.array([[11,13,12],[14,16,15],[10,18,19]]) 
print(a)

[[11 13 12]
 [14 16 15]
 [10 18 19]]


In [3]:
x = np.array([[1,2,3],[6,5,4],[9,7,8]])
print(x)

[[1 2 3]
 [6 5 4]
 [9 7 8]]


In [4]:
b = a.dot(x)
print(b)

[[197 171 181]
 [245 213 226]
 [289 243 254]]


In [5]:
for l, arr in zip(['a','x','b'],[a,x,b]):
    print('{}:\n'.format(l), arr, '\n\n')

a:
 [[11 13 12]
 [14 16 15]
 [10 18 19]] 


x:
 [[1 2 3]
 [6 5 4]
 [9 7 8]] 


b:
 [[197 171 181]
 [245 213 226]
 [289 243 254]] 




## **A** $\cdot$ **X** = **B**
## How do we get **X** if we only have **A** and **B**? 

In [6]:
lstsq?

Signature: lstsq(a, b, rcond='warn')
Docstring:
Return the least-squares solution to a linear matrix equation.

Solves the equation `a x = b` by computing a vector `x` that
minimizes the Euclidean 2-norm `|| b - a x ||^2`.  The equation may
be under-, well-, or over- determined (i.e., the number of
linearly independent rows of `a` can be less than, equal to, or
greater than its number of linearly independent columns).  If `a`
is square and of full rank, then `x` (but for round-off error) is
the "exact" solution of the equation.

Parameters
----------
a : (M, N) array_like
    "Coefficient" matrix.
b : {(M,), (M, K)} array_like
    Ordinate or "dependent variable" values. If `b` is two-dimensional,
    the least-squares solution is calculated for each of the `K` columns
    of `b`.
rcond : float, optional
    Cut-off ratio for small singular values of `a`.
    For the purposes of rank determination, singular values are treated
    as zero if they are smaller than `rcond` times the large

In [7]:
lstsq(a,b)[0]

/home/markcatalysis/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


array([[1., 2., 3.],
       [6., 5., 4.],
       [9., 7., 8.]])

## **A** $\cdot$ **X** = **B**
## Now, how do we get _**A**_ if we only have **X** and **C**? 

In [8]:
print(lstsq(x.T, b.T, rcond=None)[0])


[[11. 14. 10.]
 [13. 16. 18.]
 [12. 15. 19.]]


In [9]:
a_approx = lstsq(x.T,b.T, rcond=None)[0].T
print(a_approx)
x_approx = lstsq(a, b, rcond=None)[0]
print(x_approx)

[[11. 13. 12.]
 [14. 16. 15.]
 [10. 18. 19.]]
[[1. 2. 3.]
 [6. 5. 4.]
 [9. 7. 8.]]


## Because we know the matrices that made B, we know that A and X in the above examples should be exactly solvable! 
## That said, x_approx will be slightly off from x and a_approx from a, so if you try to check validity of the answer using the comparison method.

In [10]:
print(a==a_approx)
print(a-a_approx)
print(x-x_approx)

[[False False False]
 [False False False]
 [False False False]]
[[ 3.55271368e-15 -1.06581410e-14  3.55271368e-15]
 [ 6.75015599e-14  2.84217094e-14 -4.08562073e-14]
 [ 1.77635684e-14  7.10542736e-15 -7.10542736e-15]]
[[-3.64153152e-14 -1.68753900e-14  7.50510765e-14]
 [ 1.54543045e-13  8.88178420e-14 -2.27373675e-13]
 [-1.20792265e-13 -7.10542736e-14  1.75859327e-13]]


I prefer round numbers. Rounding to 12 decimal places out:

In [11]:
print(np.round(a-a_approx,12))

[[ 0. -0.  0.]
 [ 0.  0. -0.]
 [ 0.  0. -0.]]


Now let's compare them after some mild rounding:

In [12]:
print(a==np.round(a_approx,12))

[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]


### Why use this tool? We can use it to find approximations for factoring matrices that minimize reconstruction error.

### Say we have **W**$\cdot$**H**=**V** and we know **W** and **V** and want **H**. What's a good approximation for it?

In [13]:
V = np.random.randint(0,5,[4,6])
print(V)

[[1 3 3 3 3 1]
 [3 3 4 2 4 1]
 [0 3 4 4 0 4]
 [3 4 2 1 2 1]]


In [14]:
W = np.array([[1,0,2],[4,2,0],[3,1,2],[2,4,9]])
print(W)

[[1 0 2]
 [4 2 0]
 [3 1 2]
 [2 4 9]]


In [15]:
H = lstsq(W,V,rcond=None)[0]
print(H)

[[ 0.13227513  1.12698413  1.62433862  1.57142857  0.69312169  1.10582011]
 [ 0.95767196 -0.92063492 -1.35978836 -2.14285714  0.05820106 -1.43386243]
 [-0.12169312  0.6031746   0.46560847  0.71428571  0.04232804  0.5026455 ]]


### Interesting though. We have some negative values in this H matrix. Might be worth using the `numpy.ndarray.clip()` method in the future.